In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *
from sklearn.metrics import classification_report
import tensorflow_addons as tfa
from keras.utils import io_utils

In [2]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
test = pd.read_csv("../../dataset/goodreads_test.csv")
vocabulary2 = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)

In [3]:
train['review_text'] = train['review_text'].str.replace('[^\w\s]','')

C:\Users\Carmo\AppData\Local\Temp\ipykernel_17268\897452271.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['review_text'] = train['review_text'].str.replace('[^\w\s]','')


In [ ]:
#model_list = [cnn1,cnn2, cnn3,cnn4,cnn5,cnn6, cnn8, cnn9, cnn10]
model_list = [transformer_classifier2]#

def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
seeds = [42]
for seed in seeds:
    keras.utils.set_random_seed(seed)
    for model_obj in model_list:
        model = model_obj.Model(vocabulary2)
        model.model.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                           loss=keras.losses.categorical_crossentropy,
                           metrics=[keras.metrics.categorical_accuracy, tfa.metrics.F1Score(num_classes=6, average='weighted')]
                           )
        print(model.name)
        print(model.model.summary())
        if not os.path.exists(f"logs/{model.name}"):
            os.mkdir(f"logs/{model.name}")
        if not os.path.exists(f"checkpoint/{model.name}"):
            os.mkdir(f"checkpoint/{model.name}")
        chekpoint = keras.callbacks.ModelCheckpoint(f'checkpoint/{model.name}/', save_weights_only=True,
        monitor='val_f1_score',
        mode='max',
        save_best_only=True)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}")
        model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=4, callbacks=[keras.callbacks.LearningRateScheduler(scheduler,0)], batch_size=250, validation_split=0.2)

transformer_classifier2
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 256)         0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 token_and_position_embedding (  (None, 256, 300)    32926500    ['text_vectorization[0][0]']     
 TokenAndPositionEmbedding)                                                                       
                                                                      

In [ ]:
model.model.load_weights('checkpoint/unet1')

In [ ]:
res = model.model.predict([train['review_text'], train['n_comments'], train['n_votes']])

In [ ]:
restest = model.model.predict([test['review_text'], test['n_comments'], test['n_votes']])

In [ ]:
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
test_data = np.array(ff)

ff = []
for line in tqdm(res):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
train_data = np.array(ff)

In [ ]:
print(classification_report(train['rating'], train_data))

In [ ]:
test['rating'] = test_data

id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])
df['review_id'] = id
df['rating'] = rating

In [ ]:
df.to_csv('submission_unet2_embedding_class_weights_model.csv',index=False )


In [ ]:
model.model.save('unet2')